In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def custom_round(x, base=0.05):
    return base*round(x/base)


def get_qualSchool(df, col="qual_school"):
    maxGrad=data.groupby("qual_school").max().reset_index()
    maxGrad=maxGrad[["qual_school","avg_grad_round"]]
    max0=maxGrad["avg_grad_round"][0]
    max1=maxGrad["avg_grad_round"][1]
    max2=maxGrad["avg_grad_round"][2]
    max3=maxGrad["avg_grad_round"][3]
    df.loc[df["avg_grad_round"]<=max0,"qual_school"]=0
    df.loc[((df["avg_grad_round"]>max0) &(df["avg_grad_round"]<=max1)) ,"qual_school"]=1
    df.loc[((df["avg_grad_round"]>max1) &(df["avg_grad_round"]<=max2)) ,"qual_school"]=2
    df.loc[((df["avg_grad_round"]>max2) &(df["avg_grad_round"]<=max3)) ,"qual_school"]=3
    df.loc[df["avg_grad_round"]>max3,"qual_school"]=4
    return df
    

In [3]:
dataPrep=pd.read_csv("datasets/dataForML.csv")
data=pd.read_csv("datasets/dataForMLScatter.csv")

dataPrep["total_grads_of_cohort"]=dataPrep["total_grads_of_cohort"]/100
df_mean=dataPrep.groupby("dbn").mean()
df_mean=df_mean.reset_index()
df_mean["avg_grad_round"]=df_mean["total_grads_of_cohort"].apply(lambda x: custom_round(x))
df_mean=df_mean[["dbn","avg_grad_round"]]

stdGrad=dataPrep.groupby("dbn").std()
stdGrad=stdGrad.reset_index()
stdGrad=stdGrad[["dbn","total_grads_of_cohort"]]
stdGrad=stdGrad.rename(columns={"total_grads_of_cohort":"std_Grads"})
stdGrad["std_Grads_round"]=stdGrad["std_Grads"].apply(lambda x: custom_round(x))
data=data.merge(stdGrad, on="dbn",how="left")
data=data.merge(df_mean, on="dbn",how="left")
data=data.dropna(subset=["trees","crashes","pc","shootings","arrests","programs","bins","avg_perc Grads","std_Grads"])
data=data.sort_values(by=["avg_grad_round"])
data=data.reset_index()
data["qual_school"]=0
class_width=data.shape[0]//5
class_width
data.loc[class_width:2*class_width,"qual_school"]=1
data.loc[class_width*2:3*class_width,"qual_school"]=2
data.loc[class_width*3:4*class_width,"qual_school"]=3
data.loc[class_width*4:,"qual_school"]=4



In [4]:


X=data[["trees","crashes","pc","shootings","arrests","programs","bins"]]


y=data[ "qual_school"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1, stratify=y, random_state=42)


In [ ]:
from sklearn.metrics import f1_score
kneighbours_test=[]
score_l_test=[]
for i in range(1,20):
    neigh=KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train,y_train)
    pred=neigh.predict(X_test)
    score=f1_score(y_test,pred, average="macro")
    kneighbours_test.append(i)
    score_l_test.append(score)


In [ ]:
list(zip(score_l_test,kneighbours_test))

In [5]:
import pickle
final_model=KNeighborsClassifier(n_neighbors=9)
final_model.fit(X_train,y_train)

pickle.dump(final_model,open("final_model.sav","wb"))

In [ ]:
plt.scatter(kneighbours_test,score_l_test)

In [7]:
model=pickle.load(open("final_model.sav","rb"))

def get_pred(features):
    pred=model.predict(features)


    return pred

In [23]:
get_pred(x)

array([2])

In [21]:
x=X_train[:1]

In [22]:
x

,trees,crashes,pc,shootings,arrests,programs,bins
231,-0.145996,-1.103344,0.688072,-0.045086,-0.84546,-0.063023,-0.539355
